In [1]:
import os
os.chdir('..')

In [2]:
%load_ext autoreload
%autoreload 2

In [37]:
pip install staticconf

Looking in indexes: https://pypi.python.org/simple/
ERROR: Could not find a version that satisfies the requirement staticconf
ERROR: No matching distribution found for staticconf
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'staticconf'

In [3]:
import torch
import torchvision
from pytorch_lightning.callbacks import ModelCheckpoint
from src.model.lit_module import LitModule
from src.data.dataset import VideoLabelDataset
import src.constants as const
from torch.utils.data import DataLoader
from src.data.dataset import (VideoLabelDataset,
                              VideoFolderPathToTensor,
                              VideoResize)
import plotly
import numpy as np
import pandas as pd

In [4]:
dataset = VideoLabelDataset(
            const.LABELS_TABLE_QA_PATH,
            img_transform=torchvision.transforms.Compose([
                VideoFolderPathToTensor(),
                VideoResize(const.IMG_SIZE)]))

In [5]:
dataloader = DataLoader(dataset, batch_size=100, num_workers=6)

In [31]:
from src.model.lit_module import LitModule

In [39]:
import yaml

In [44]:
with open("lightning_logs/version_11/hparams.yaml", 'r') as stream:
    hparams = yaml.load(stream, Loader=yaml.Loader)

In [45]:
hparams

{'accelerator': 'ddp',
 'accumulate_grad_batches': 1,
 'amp_backend': 'native',
 'amp_level': 'O2',
 'auto_lr_find': False,
 'auto_scale_batch_size': False,
 'auto_select_gpus': False,
 'automatic_optimization': None,
 'batch_size': 32,
 'benchmark': False,
 'beta': 0.001,
 'check_val_every_n_epoch': 1,
 'checkpoint_callback': True,
 'dec_hidden_size': 32,
 'dec_num_hidden_layers': 10,
 'dec_num_question_inputs': 1,
 'dec_single_answer_dim': 1,
 'default_root_dir': None,
 'deterministic': False,
 'distributed_backend': None,
 'dl_num_workers': 8,
 'enable_pl_optimizer': False,
 'enc_dr_rate': 0.0,
 'enc_fixed_cnn_weights': False,
 'enc_pretrained': True,
 'enc_rnn_hidden_dim': 32,
 'enc_rnn_num_layers': 2,
 'fast_dev_run': False,
 'filt_initial_log_var': -10.0,
 'flush_logs_every_n_steps': 100,
 'gpus': 1,
 'gradient_clip_val': 0,
 'learning_rate': 0.0005,
 'limit_test_batches': 1.0,
 'limit_train_batches': 1.0,
 'limit_val_batches': 1.0,
 'log_every_n_steps': 50,
 'log_gpu_memory': No

In [46]:
lit_mod = LitModule(**hparams)
val_dataloader = lit_mod.val_dataloader()

In [47]:
videos, questions, answers, hidden_states, vid_folder  = iter(val_dataloader).next()

In [48]:
videos.shape

torch.Size([32, 3, 11, 224, 224])

In [49]:
ll lightning_logs/version_11/checkpoints/

total 295136
-rw-r--r--  1 henrik  staff  135112663 Jan  9 23:54 epoch=999-step=29999.ckpt


In [50]:
checkpoint_path = './lightning_logs/version_11/checkpoints/epoch=999-step=29999.ckpt'

In [51]:
model = LitModule.load_from_checkpoint(checkpoint_path)

In [52]:
hidden_states_pred = model.eval()(videos)
answers_pred = [dec(hidden_states_pred, questions) for dec in model.decoding_agents]
answers_pred = torch.cat(answers_pred, axis=1)

In [53]:
df_answers_pred = pd.DataFrame(answers_pred.detach().numpy(), columns=[f'{c}_pred' for c in const.ANSWER_COLS])
df_hidden_states_pred = pd.DataFrame(hidden_states_pred.detach().numpy(), columns=[f'{c}_pred' for c in const.HIDDEN_STATE_COLS])
df_answers = pd.DataFrame(answers.detach().numpy(), columns=const.ANSWER_COLS)
df_hidden_states = pd.DataFrame(hidden_states.detach().numpy(), columns=const.HIDDEN_STATE_COLS)

In [54]:
import plotly.graph_objects as go
fig = go.Figure()
for c in const.ANSWER_COLS:
    fig.add_trace(go.Scatter(x=df_answers[c], y=df_answers_pred[f'{c}_pred'],
                        mode='markers',
                        name=c))
fig.show()

In [55]:
fig = go.Figure()
for c in const.HIDDEN_STATE_COLS:
    fig.add_trace(go.Scatter(x=df_hidden_states[c], y=df_hidden_states_pred[f'{c}_pred'],
                        mode='markers',
                        name=c))
fig.show()

In [30]:
hidden_states

tensor([[ 378.,  937.,   49.],
        [ 407.,  929.,   31.],
        [ 309.,  926.,   41.],
        [ 352., 1099.,   32.],
        [ 431.,  979.,   28.],
        [ 698.,  995.,   26.],
        [ 319.,  975.,   27.],
        [ 644.,  919.,   43.],
        [ 495.,  941.,   42.],
        [ 375.,  973.,   35.],
        [ 316.,  964.,   46.],
        [ 680., 1016.,   29.],
        [ 338., 1083.,   35.],
        [ 541., 1024.,   46.],
        [ 652., 1095.,   41.],
        [ 574., 1031.,   25.],
        [ 568.,  996.,   33.],
        [ 560.,  908.,   28.],
        [ 608., 1068.,   27.],
        [ 416.,  901.,   37.],
        [ 688., 1049.,   33.],
        [ 400., 1010.,   42.],
        [ 352.,  927.,   35.],
        [ 672.,  969.,   27.],
        [ 410., 1048.,   31.],
        [ 696., 1063.,   41.],
        [ 759.,  947.,   49.],
        [ 647., 1088.,   45.],
        [ 305., 1069.,   45.],
        [ 476.,  994.,   27.],
        [ 486.,  921.,   48.],
        [ 653.,  950.,   32.],
        